In this note, I apply __L1-based__ feature selection. Note that due to its sparsed representaiton of the model, L1-based feature selectin is widely used.

To make data suitable for logistic regression (LR) algorithm, I have applied mean 0 and standard deviation for 'age' column, and midrange 0 and range 2 (i.e., minimum -1 and maximum 1) for other columns, inlcuding date time variables and all agreation (occurance counting) variables.

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
#from xgboost.sklearn import XGBClassifier

np.random.seed(0)

from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
import sklearn
import sklearn.grid_search
#from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold

import math

from NDCG_score_func import ndcg_scorer

Loading X,y and X_test

In [2]:
X= np.load('X.npy')
y= np.load('y.npy')
X_test= np.load('X_test.npy')

### Feature selection using logistic regression regularized with L1 norm

- Feature selection

In [3]:
%%time
#Using grid function to do k-fold cross validation....
#for each set of parameters, run K-fold cross validation
C_iter =[0.1] #budget paramter.
parameters = {'C':C_iter}
LgR = sklearn.linear_model.LogisticRegression(penalty='l1') #n_jobs=-1
clf = sklearn.grid_search.GridSearchCV(LgR, parameters, cv=2, n_jobs=1)
clf.fit(X,y)
a = clf.grid_scores_
a

Wall time: 17min 17s


In [4]:
a

[mean: 0.64019, std: 0.00041, params: {'C': 0.1}]

Selecting from model: Using the best fitted estimator to perform feature selection:

In [5]:
# clf.best_estimator_ 

In [18]:
clf.best_estimator_ .coef_.shape

(12L, 226L)

In [7]:
X.shape

(213451L, 226L)

In [8]:
from sklearn.feature_selection import SelectFromModel
lg = clf.best_estimator_ 
model = SelectFromModel(lg, prefit=True)
X_new = model.transform(X)
X_new.shape

(213451L, 122L)

In [9]:
X_test_new = model.transform(X_test)
X_test_new.shape

(62096L, 122L)

In [10]:
X.shape

(213451L, 226L)

### Performance using __random forest__ with selected features

In [11]:
%%time
#select m features out of p features randomly for each node
#bootstropped used to boost the randomness of the each tree
rf = RandomForestClassifier(max_features=35, n_estimators=120, n_jobs=1, min_samples_split=5)#n_jobs=2 #, min_samples_split=5  max_features=12
#view the whole data set as trainning set, and cross validate using K-fold cross validata(K=10)
kf = KFold(len(X_new), n_folds=5, random_state=42)
score_list = cross_val_score(rf, X_new, y, cv=kf,scoring=ndcg_scorer)# scoring=ndcg_scorer
print score_list
print np.mean(score_list)

[ 0.82005258  0.81832695  0.81880954  0.81861809  0.81940185]
0.81904180069
Wall time: 19min 26s


Summary: 
- 2 fold L1 C=0.01 0.1, 1, 10  NDCG=0.6356 0.64019,0.64115 ,0.64103,
- selected feature:71 122, 177, 209
- random forest with selected features:0.8180, 0.81904, 0.819467, 0.8195
- test performance:0.87949 (rank 313) 0.88098 0.88077 0.88135 (rank 300?)
- RF paramters: min_samples_split= 27,35,40,40

Remark: 
- very similar in terms of performance. 
- Thourough cross validation for tunning RF parameters is not done, e.g., max_features. The used parameters are 35, 35,40 repectively. 

### Generating submission using the appraoch above

In [12]:
# #get some useful paramters ready to use
# df_train = pd.read_csv('train_users_2.csv')
# df_test = pd.read_csv('test_users.csv')
# labels = df_train['country_destination'].values
# df_train = df_train.drop(['country_destination'], axis=1)


# id_test = df_test['id']
# piv_train = df_train.shape[0]

# le = LabelEncoder()
# le.fit(labels) #is this  integer encoding a must-do or optional?

LabelEncoder()

In [13]:
# #set up classifier parameters
# #number of n_estimators trees 
# rf = RandomForestClassifier(n_estimators = 120,max_features = 35, min_samples_split=5, n_jobs=-1)
# rf.fit(X_new,y)
# y_pred = rf.predict_proba(X_test_new)

# #Taking the 5 classes with highest probabilities
# ids = []  #list of ids
# cts = []  #list of countries
# for i in range(len(id_test)):
#     #idx:internal variable 
#     idx = id_test[i] 
#     #dupliccate ids 5 times and put it to the list ids
#     ids += [idx] * 5
#     cts += le.inverse_transform(np.argsort(y_pred[i])[::-1])[:5].tolist()
    
# #Generate submission
# sub = pd.DataFrame(np.column_stack((ids, cts)), columns=['id', 'country'])
# sub.to_csv('sub5.csv',index=False)
# sub.head()

,id,country
0,5uwns89zht,NDF
1,5uwns89zht,US
2,5uwns89zht,IT
3,5uwns89zht,other
4,5uwns89zht,PT


0.88135 obtained similar to random forest selection

- Next, I will try C=1 to check if more features will be ruled out and is the performance going to be changed. (Notice that I need to save the relavant variables for selected data and coffiencienit for later use regarding RF CV and selected feature inspection)

- (later)Also, L_2 can also be tried as L2 has advantage in sklearn faster implementation. One can use a pretuned threshold to implement this approache using a for loop cross-validation.

Conclusion: It is well known that Logistic regression (LR) is usually not suitable multi-class classification task, especially for imbalanced data set. As shown above, the accuacy is much poorer than Random Forest. But still we can use is a approach to do feature selection. 